In [ ]:
import mujoco
import numpy as np
import stac_mjx
from pathlib import Path
import h5py
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import re
import time
import imageio
import mediapy as media
import os
from jax import numpy as jp
import hydra
from omegaconf import DictConfig, OmegaConf

OmegaConf.register_new_resolver("eq", lambda x, y: x.lower() == y.lower())
OmegaConf.register_new_resolver("contains", lambda x, y: x.lower() in y.lower())
OmegaConf.register_new_resolver(
    "resolve_default", lambda default, arg: default if arg == "" else arg
)

In [ ]:
# glfw is faster, but use osmesa if glfw not available
%env MUJOCO_GL=glfw
%env PYOPENGL_PLATFORM=glfw

# Enable XLA flags if on GPU
stac_mjx.enable_xla_flags()

base_path = Path.cwd()


# Load configs
cfg = stac_mjx.load_configs(base_path / "../configs")

kp_data, sorted_kp_names = stac_mjx.load_mocap(cfg, base_path)

In [ ]:
# Run stac
fit_path, ik_only_path = stac_mjx.run_stac(
 cfg,
 kp_data, 
 sorted_kp_names, 
 base_path=base_path
)

In [ ]:
data_path = base_path / "1500f_stick_fit_offsets.h5"
n_frames = 1500
save_path = base_path / "../videos/1500f_render_stick_bug.mp4"

# Call mujoco_viz
cfg, frames = stac_mjx.viz_stac(
    data_path=data_path,
    n_frames=n_frames,
    save_path=save_path,
    start_frame=0,
    camera="side_alt",
    base_path=Path.cwd(),
    width=1920,
    height=1200
)

# Show the video in the notebook (it is also saved to the save_path)
media.show_video(frames, fps=cfg.model.RENDER_FPS)